In [ ]:
# Transformation
for column in numerical_list:
    df[column +'_sqrt']=np.sqrt(1 + df[column].values).astype(float)
    oot[column +'_sqrt']=np.sqrt(1 + oot[column].values).astype(float)
    
for column in numerical_list:
    df[column +'_log']=np.log2(1 + df[column].values).astype(float)
    oot[column +'_log']=np.log2(1 + oot[column].values).astype(float) 
    
for column in numerical_list:
    df[column +'_inv']=(1 /(1+df[column].values)).astype(float)
    oot[column +'_inv']=(1 /(1+oot[column].values)).astype(float)

  
oot=oot.replace([np.inf, -np.inf], np.nan)
df=df.replace([np.inf, -np.inf], np.nan)

In [ ]:
# Train Test Split and outlier Treatment

from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.15, shuffle=True, stratify=y, random_state=10001)

for col in numerical_list:
    percentiles = train_x[col].quantile([0.01,0.99]).values
    train_x[col][train_x[col] <= percentiles[0]] = percentiles[0]
    train_x[col][train_x[col] >= percentiles[1]] = percentiles[1]

    valid_x[col][valid_x[col] <= percentiles[0]] = percentiles[0]
    valid_x[col][valid_x[col] >= percentiles[1]] = percentiles[1]    
    
    oot1[col][oot1[col] <= percentiles[0]] = percentiles[0]
    oot1[col][oot1[col] >= percentiles[1]] = percentiles[1]

In [ ]:
# Missing Value Treatment
mtreat=[]
for column in mtreat:
    train_x[column]=train_x[column].fillna(train_x[column].mean())  
    valid_x[column]=valid_x[column].fillna(train_x[column].mean()) 
    oot1[column]=oot1[column].fillna(train_x[column].mean())

In [ ]:
# Date Part Variables
import pandas as pd
import numpy as np
import os
import re


date_part_data = df

#input Data Frame
#Date Variable
#Unique ID
#time = True/False(Date Variable has time stamp or not)
#start_end= True/False(if True it will add month start, Month End, year_start, year_end fratures)

def DatePart(df, DATE,UID,time=False,start_end=True):
    df_output=df[[DATE,UID]]
    if isinstance(DATE,str): 
        DATE = [DATE]
    for date_part_name in DATE:
        fld = df_output[date_part_name]
        date_part_dtype = fld.dtype
        if isinstance(date_part_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            date_part_dtype = np.datetime64
            
        if not np.issubdtype(date_part_dtype, np.datetime64):
                df_output[date_part_name] = fld = pd.to_datetime(fld, infer_datetime_format=True)
        DateColName =date_part_name
        levl = ['Quarter','Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','Weekday']
        start_end=['Is_month_end', 'Is_month_start', 'Is_quarter_end','Is_quarter_start', 'Is_year_end', 'Is_year_start','Is_Leap_Year']
        if time: levl = levl + ['Hour', 'Minute', 'Second']
        if start_end: levl = levl + start_end
        for n in levl: df_output[DateColName +"_"+ n] = getattr(fld.dt, n.lower()).astype(int)
        df_output.drop(date_part_name, axis=1, inplace=True)
        df_output[DateColName +"_"+'Is_Weekend'] = [1 if value > 4  else 0 for value in df_output[DateColName +"_"+'Day']]
        df_output[DateColName +"_"+'Is_WeekStart'] = [1 if value < 2  else 0 for value in df_output[DateColName +"_"+'Day']]
        df_output=pd.merge(df,df_output,on=UID,how='inner')
    return df_output
        
Date_Variable_Data=DatePart(date_part_data,'date','uid',time=True,start_end=True)


In [ ]:
# Aggregation Variables

cat=['gender','network',]
num=['age','income']


uid='recid'



gc.collect()


def agg(num_col, num_rows = None, nan_as_category = True):
    agg_data = df1
    aggregations = {
                num_col:['mean','std','max','min','skew','median']                        
                }
    agg_data_agg = agg_data.groupby(column).agg(aggregations)
    agg_data_agg.columns = ([column +"_"+ e[0] + "_" + e[1] for e in agg_data_agg.columns.tolist()])
    gc.collect()
    return agg_data_agg



for column in cat:
    for column1 in num:
        agg_data =agg(column1)
#        print(agg_data.columns)

        df1 = pd.merge(agg_data, df1, on = [column], how = 'left')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

from sklearn.metrics import confusion_matrix, brier_score_loss, accuracy_score

def show_all(df, nrow=None, ncol=None):
    with pd.option_context('display.max_rows', nrow, 'display.max_columns', ncol):
        display(df)
        
def plot_roc(actual, predicted, fig_name='roc_curve'):
	fpr, tpr, threshold = metrics.roc_curve(actual, predicted)
	roc_auc = metrics.auc(fpr, tpr)
	plt.clf()
	plt.plot(fpr,tpr,color='red',label='AUC=%0.2f'%roc_auc)
	plt.plot([0,1],[0,1],'k')
	plt.xlim([0.0,1.0])
	plt.ylim([0.0,1.0])
	plt.grid()
	plt.xlabel('False Positive Rate')
	plt.ylabel('True Positive Rate')
	titleStr='ROC Curve'
	plt.title(titleStr)
	plt.legend(loc="lower right")
	plt.savefig(fig_name)
    
def get_decile_table(df, dv='dv', score='score'):
	df = df[[dv, score]].copy()
	df.sort_values(score, ascending=False, inplace = True)
	df['Decile'] = pd.qcut(np.arange(1, df.shape[0]+1, 1), 10, labels=False)
	df['Decile'] = df['Decile'] + 1
	brier_losses = df.groupby(['Decile']).apply(lambda x: brier_score_loss(x[dv], x[score]))
	
	decl_tbl = pd.crosstab(df['Decile'], df[dv])
	
	if 0 not in decl_tbl.columns.tolist():
		decl_tbl[0] = 0
		decl_tbl = decl_tbl[[0, 1]]
	elif 1 not in decl_tbl.columns.tolist():
		decl_tbl[1] = 0

	decl_tbl.columns = ['Non_Event', 'Event']
	decl_tbl.reset_index(inplace=True)
	decl_tbl = decl_tbl[['Decile', 'Event', 'Non_Event']]
	decl_tbl['Total'] = decl_tbl['Event'] + decl_tbl['Non_Event']
	decl_tbl['Cum_Event'] = np.cumsum(decl_tbl['Event'])
	decl_tbl['Cum_Non_Event'] = np.cumsum(decl_tbl['Non_Event'])
	decl_tbl['Cum_Total'] = np.cumsum(decl_tbl['Total'])
	decl_tbl['Cum_Event_Prop'] = decl_tbl['Cum_Event']/decl_tbl['Cum_Event'].max()
	decl_tbl['Cum_Non_Event_Prop'] = decl_tbl['Cum_Non_Event']/decl_tbl['Cum_Non_Event'].max()
	decl_tbl['Cum_Lift_Event'] = decl_tbl['Cum_Event_Prop']/(decl_tbl['Decile']/10)
	decl_tbl['KS'] = decl_tbl['Cum_Event_Prop'] - decl_tbl['Cum_Non_Event_Prop']
	decl_tbl['Brier_Score_Loss'] = brier_losses
	return decl_tbl


def get_decile_lift_curve(decl_tbl=None, df=None, dv='dv', score='score'):
	if decl_tbl is None:
		decl_tbl = get_decile_table(df, dv, score)
		
	return decl_tbl['Decile'].values, decl_tbl['Cum_Lift_Event'].values    



def plot_decile_lift_curve(decl_tbl=None, df=None, dv='dv', score='score', fig_name="decile_lift"):
	deciles, lifts = get_decile_lift_curve(decl_tbl=decl_tbl, df=df, dv=dv, score=score)
	plt.clf()
	plt.plot(deciles, lifts)
	plt.xlabel("Deciles")
	plt.ylabel("Cum Lift of an Event")
	plt.title("Decile Lift")
	plt.savefig(fig_name)
    
def get_confusion_matrix(df, prob_thresh = 0.5, labels=[0, 1], dv='dv', score='score'):
	con_mat = confusion_matrix(df[dv], df[score] > prob_thresh, labels=labels)
	return con_mat


plot_roc(jul_19['life_lead'], jul_19['prediction'], 'jul_19')



x = get_confusion_matrix(jul_19, prob_thresh =0.33, labels=[0, 1], dv='life_lead', score='prediction')
print(x)
print(x.sum(axis=1), x.sum(axis=0), x.sum())

In [ ]:
#groupby
temp = df1.groupby(by = 'homeno').agg({'life_sms' : 'sum'})
temp2 = pd.DataFrame()
temp2['homeno'] = temp.index
temp2['sms_cnt_workno'] = temp.life_sms.values
gc.collect()

In [ ]:
act_17=alls[alls['DOL'] >= '2018-01-01'] ;gc.collect()
act_17_1=act_17[act_17['DOJ'] < '2018-01-01'];gc.collect()
act_17_1['Process_Type']=act_17_1['Process_Type'].str.upper()
act_17_1['Pay_Code']=act_17_1['Pay_Code'].str.upper()
act_17_2=act_17_1[act_17_1['Pay_Code'] != 'GRATUITY'] ;gc.collect()
act_17_3=act_17_2[act_17_2['Process_Type'] != 'FANDF'] ;gc.collect()
july_1 = pd.to_datetime('2018-07-01', format = '%Y-%m-%d')
gc.collect()

In [ ]:
from pandasql import *
pysqldf = lambda r: sqldf(r, globals())
r  =""" select """
act_17_uni = pysqldf(r)
gc.collect()

In [1]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=12607 sha256=1727ad956a7737da314d6b1f15b1eb5fcab6488e960263df922d918503ca64c8
  Stored in directory: /home/lokeshstat/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#  Libraries
import numpy as np 
import pandas as pd 
# Data processing, metrics and modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold,KFold
from bayes_opt import BayesianOptimization
from datetime import datetime
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc
from sklearn import metrics
from sklearn import preprocessing
# Lgbm
import lightgbm as lgb
# Suppr warning
import warnings
warnings.filterwarnings("ignore")

import itertools
from scipy import interp

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')


train_df=X_train
test_df=X_test

features=trainOn

target = target



#cut tr and val
bayesian_tr_idx, bayesian_val_idx = train_test_split(train_df[features], test_size = 0.2, random_state = 42, stratify = train_df[target])
bayesian_tr_idx = bayesian_tr_idx.index
bayesian_val_idx = bayesian_val_idx.index


#categorical_list = []
#numerical_list = []
#for i in train_df.columns.tolist():
#    if train_df[i].dtype=='object':
#        categorical_list.append(i)
#   
#print('Number of categorical features:', str(len(categorical_list)))
#print('Number of numerical features:', str(len(numerical_list)))



#black box LGBM 
def LGB_bayesian(
    learning_rate,
    num_leaves, 
    bagging_fraction,
    feature_fraction,
    min_child_samples, 
    min_child_weight,
    subsample, 
    min_data_in_leaf,
    max_depth,
    colsample_bytree,
    reg_alpha,
    reg_lambda,
    scale_pos_weight
     ):
    
    # LightGBM expects next three parameters need to be integer. 
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int
    

    param = {
              'num_leaves': num_leaves, 
              'min_child_samples': min_child_samples, 
              'min_data_in_leaf': min_data_in_leaf,
              'min_child_weight': min_child_weight,
              'bagging_fraction' : bagging_fraction,
              'feature_fraction' : feature_fraction,
              'subsample': subsample, 
              'max_depth': max_depth,
              'colsample_bytree': colsample_bytree,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight':scale_pos_weight,
              'objective': 'binary',
              'save_binary': True,
              'seed': 1337,
              'feature_fraction_seed': 1337,
              'bagging_seed': 1337,
              'drop_seed': 1337,
              'data_random_seed': 1337,
              'boosting_type': 'gbdt',
              'verbose': 1,
#              'is_unbalance': False,
              'boost_from_average': True,
              'metric':'auc'}    
    
    oof = np.zeros(len(train_df))
    categorical = ['app_code','hour', 'day']

    trn_data= lgb.Dataset(train_df.iloc[bayesian_tr_idx][features].values, label=train_df.iloc[bayesian_tr_idx][target].values)
    val_data= lgb.Dataset(train_df.iloc[bayesian_val_idx][features].values, label=train_df.iloc[bayesian_val_idx][target].values)

    clf = lgb.train(param, trn_data,  num_boost_round=100000, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 50)
    
    oof[bayesian_val_idx]  = clf.predict(train_df.iloc[bayesian_val_idx][features].values, num_iteration=clf.best_iteration)  
    
    score = roc_auc_score(train_df.iloc[bayesian_val_idx][target].values, oof[bayesian_val_idx])

    return score




# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (2, 50), 
    'min_data_in_leaf': (5, 150),
    'bagging_fraction' : (0.1,0.9),
    'feature_fraction' : (0.1,0.9),
    'learning_rate': (0.01, 0.07),
    'min_child_weight': (0.00001, 0.01),   
    'min_child_samples':(10, 500), 
    'subsample': (0.2, 0.8),
    'colsample_bytree': (0.4, 0.8), 
    'reg_alpha': (1, 10), 
    'reg_lambda': (1, 10),
    'max_depth':(-1,15),
    'scale_pos_weight':(1,100)
}




LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=42)



print(LGB_BO.space.keys)




init_points = 15
n_iter = 10




print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)




LGB_BO.max['target']





LGB_BO.max['params']

NameError: name 'train' is not defined

In [ ]:
X = oot[numerical_list]
y = oot[dv]

from sklearn.feature_selection import SelectFromModel
train_x, valid_x, train_y, valid_y = train_test_split(X[numerical_list], y, test_size=0.1, shuffle=True, stratify=y, random_state=9001)
from xgboost import XGBClassifier
xgbc=XGBClassifier(n_estimators=10000, 
                   learning_rate=0.01, 
                   num_leaves=2, 
                   colsample_bytree=1,
                   reg_alpha=0, 
                   reg_lambda=50, 
                   min_split_gain=0.5, 
                   min_child_weight=30)

xgbc.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)
xgb_importance_df = pd.DataFrame()
xgb_importance_df["feature"] = numerical_list
xgb_importance_df["importance"] = xgbc.feature_importances_*(100)

xgbc.best_iteration

xgb_importance_df.to_excel('fi_xgb_oot.xlsx',index=False)


sub_preds = np.zeros(df.shape[0])
sub_preds = xgbc.predict_proba(df[numerical_list], xgbc.best_iteration)[:, 1] 
df['score'] = sub_preds